In [1]:
import torch
import transformers
from ast import literal_eval
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM, SFTConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import Dataset
import json
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import evaluate
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
from peft import AutoPeftModelForCausalLM, LoraConfig

pd.set_option('display.max_columns', None)

In [2]:
# 난수 고정
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

set_seed(42) # magic number :)

In [3]:
# Load the train dataset
# TODO Train Data 경로 입력
dataset = pd.read_csv('train.csv') 

# Flatten the JSON dataset
records = []
for _, row in dataset.iterrows():
    problems = literal_eval(row['problems'])
    record = {
        'id': row['id'],
        'paragraph': row['paragraph'],
        'question': problems['question'],
        'choices': problems['choices'],
        'answer': problems.get('answer', None),
        "question_plus": problems.get('question_plus', None),
    }
    # Include 'question_plus' if it exists
    if 'question_plus' in problems:
        record['question_plus'] = problems['question_plus']
    records.append(record)
        
# Convert to DataFrame
df = pd.DataFrame(records)

In [4]:
df

,id,paragraph,question,choices,answer,question_plus
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"[ㄱ, ㄴ, ㄱ, ㄷ, ㄴ, ㄹ, ㄷ, ㄹ]",2,None
1,generation-for-nlp-426,"(가)은/는 의병계열과 애국계몽 운동 계열의 비밀결사가 모여 결성된 조직으로, 총사...",(가)에 대한 설명으로 옳지 않은 것은?,"[고려 문종 때에 남경(南京)으로 승격되었다., 종루(鐘樓), 이현, 칠패 등에서 ...",1,None
2,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,(가) 지역에 대한 설명으로 옳은 것은?,"[이곳에 대장도감을 설치하여 재조대장경을 만들었다., 지눌이 이곳에서 수선사 결사운...",4,None
3,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,밑줄 친 ‘그’에 대한 설명으로 옳은 것은?,"[살수에서 수의 군대를 물리쳤다 ., 김춘추 의 신라 왕위 계승을 지원하였다 ., ...",2,None
4,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",(가) 인물이 추진한 정책으로 옳지 않은 것은?,"[사창제를 실시하였다 ., 대전회통을 편찬하였다 ., 비변사의 기능을 강화하였다 ....",3,None
...,...,...,...,...,...,...
2026,generation-for-nlp-2893,"“헐값에 팔리는 냉동 오렌지주스만 잔뜩 사가고, 쿠폰을 오려 모았으며, 구멍 난 스...",잭 맥도널드가 남긴 유산의 총액은 얼마인가?,"[1억8760만달러, 5000만달러, 2억달러, 1억달러, 3억달러]",1,None
2027,generation-for-nlp-2894,"넷기어코리아(지사장 김진겸, 이하 넷기어)가 뮤럴 캔버스를 가지고 넷기어 SNS에서...",넷기어가 모집하는 '뮤럴 공식 도슨트'의 주요 역할은 무엇인가?,"[전시물 설명 및 안내, 고객 서비스 제공, 제품 판매 촉진, 온라인 강좌 콘텐츠 ...",4,None
2028,generation-for-nlp-2895,서울 성동구 옥수동과 금호동은 맞붙어 있는 동네지만 아파트값은 같은 면적에서 1억원...,옥수동과 금호동의 아파트값 차이가 발생한 주된 이유는 무엇인가?,"[옥수동의 재개발이 먼저 이루어졌다, 금호동의 인구가 더 많다, 옥수동이 더 많은 ...",1,None
2029,generation-for-nlp-2896,방하남 고용노동부 장관(사진)이 취임 후 첫 외부 행사로 5년 만에 일자리를 3.5...,방하남 고용노동부 장관이 방문한 기업의 이름은 무엇인가?,"[셀트리온, 삼성전자, LG화학, 한화그룹, 현대자동차]",1,None


In [7]:
print(df.columns)  # 열 이름 확인
print(df['choices'].head())  # choices 열의 첫 몇 개 항목 확인
print(df.dtypes)  # 각 열의 데이터 타입 확인

Index(['id', 'paragraph', 'question', 'choices', 'answer', 'question_plus'], dtype='object')
0                             [ㄱ, ㄴ, ㄱ, ㄷ, ㄴ, ㄹ, ㄷ, ㄹ]
1    [고려 문종 때에 남경(南京)으로 승격되었다., 종루(鐘樓), 이현, 칠패 등에서 ...
2    [이곳에 대장도감을 설치하여 재조대장경을 만들었다., 지눌이 이곳에서 수선사 결사운...
3    [살수에서 수의 군대를 물리쳤다 ., 김춘추 의 신라 왕위 계승을 지원하였다 ., ...
4    [사창제를 실시하였다 ., 대전회통을 편찬하였다 ., 비변사의 기능을 강화하였다 ....
Name: choices, dtype: object
id               object
paragraph        object
question         object
choices          object
answer            int64
question_plus    object
dtype: object


In [ ]:
import pandas as pd
from ast import literal_eval

# choices 열을 리스트로 변환
df['choices'] = df['choices'].apply(literal_eval)

# paragraph와 question이 동일한 그룹 생성
grouped = df.groupby(['paragraph', 'question'])

# 각 그룹 내에서 choices가 다른 경우 찾기
different_choices = []

for (para, ques), group in grouped:
    if len(group) > 1 and not group['choices'].apply(tuple).nunique() == 1:
        different_choices.append(group)

# 결과 데이터프레임 생성
result_df = pd.concat(different_choices) if different_choices else pd.DataFrame()

# 결과 출력
if not result_df.empty:
    print("paragraph와 question이 같지만 choices가 다른 경우:")
    print(result_df[['id', 'paragraph', 'question', 'choices', 'answer', 'question_plus']])
    print(f"\n총 {len(result_df)} 개의 행이 발견되었습니다.")
else:
    print("paragraph와 question이 같지만 choices가 다른 경우가 없습니다.")

In [9]:
import pandas as pd

# paragraph와 question이 동일한 그룹 생성
grouped = df.groupby(['paragraph', 'question'])

# 각 그룹 내에서 choices가 다른 경우 찾기
different_choices = []

for (para, ques), group in grouped:
    if len(group) > 1 and group['choices'].apply(str).nunique() > 1:
        different_choices.append(group)

# 결과 데이터프레임 생성
result_df = pd.concat(different_choices) if different_choices else pd.DataFrame()

# 결과 출력
if not result_df.empty:
    print("paragraph와 question이 같지만 choices가 다른 경우:")
    print(result_df[['id', 'paragraph', 'question', 'choices', 'answer', 'question_plus']])
    print(f"\n총 {len(result_df)} 개의 행이 발견되었습니다.")
else:
    print("paragraph와 question이 같지만 choices가 다른 경우가 없습니다.")

paragraph와 question이 같지만 choices가 다른 경우:
                         id  \
32   generation-for-nlp-463   
60   generation-for-nlp-498   
31   generation-for-nlp-461   
47   generation-for-nlp-482   
1    generation-for-nlp-426   
10   generation-for-nlp-436   
323  generation-for-nlp-819   
384  generation-for-nlp-884   
20   generation-for-nlp-449   
67   generation-for-nlp-507   

                                             paragraph  \
32   (가)농민군이 정부와 전주화약을 맺었다. (나) 농민군이 우금치에서 전투를 벌였다....   
60   (가)농민군이 정부와 전주화약을 맺었다. (나) 농민군이 우금치에서 전투를 벌였다....   
31   (가)신라의 우산국 복속 (나)고구려의 서 안평 점령(다) 백제의 대야성 점령 (라...   
47   (가)신라의 우산국 복속 (나)고구려의 서 안평 점령(다) 백제의 대야성 점령 (라...   
1    (가)은/는 의병계열과 애국계몽 운동 계열의 비밀결사가 모여 결성된 조직으로, 총사...   
10   (가)은/는 의병계열과 애국계몽 운동 계열의 비밀결사가 모여 결성된 조직으로, 총사...   
323  시나리오: 후반기 자신의 고전적 조건화 실험에서 파블로프(Ivan Pavlov)의 ...   
384  시나리오: 후반기 자신의 고전적 조건화 실험에서 파블로프(Ivan Pavlov)의 ...   
20   이제 살펴보건대, 신라가 주․군을 설치할 때 그 전정(田丁), 호구(戶口)가 현의 ...   
67   이제 살펴보건대, 신라가 주․군을 설치할 때 그 전정(田丁

In [11]:
result_df

,id,paragraph,question,choices,answer,question_plus
32,generation-for-nlp-463,(가)농민군이 정부와 전주화약을 맺었다. (나) 농민군이 우금치에서 전투를 벌였다....,이를 시기 순으로 바르게 나열한 것은?,"[(다)－(가)－(라)－(나), (다)－(라)－(나)－(가), (라)－(가)－(나)...",4,None
60,generation-for-nlp-498,(가)농민군이 정부와 전주화약을 맺었다. (나) 농민군이 우금치에서 전투를 벌였다....,이를 시기 순으로 바르게 나열한 것은?,"[국가에서 저화를 만들어 유통하였다 ., 강경포 , 원산포 등의 포구들이 상업의 중...",1,None
31,generation-for-nlp-461,(가)신라의 우산국 복속 (나)고구려의 서 안평 점령(다) 백제의 대야성 점령 (라...,다음 사건을 시기 순으로 바르게 나열한 것은?,"[(가)→(나)→(다)→(라), (가)→(라)→(나)→(다), (나)→(가)→(라)...",3,None
47,generation-for-nlp-482,(가)신라의 우산국 복속 (나)고구려의 서 안평 점령(다) 백제의 대야성 점령 (라...,다음 사건을 시기 순으로 바르게 나열한 것은?,"[대동여지도는 거리를 알 수 있도록 10리마다 눈금을 표시하였다 ., 혼일 강리역 ...",2,None
1,generation-for-nlp-426,"(가)은/는 의병계열과 애국계몽 운동 계열의 비밀결사가 모여 결성된 조직으로, 총사...",(가)에 대한 설명으로 옳지 않은 것은?,"[고려 문종 때에 남경(南京)으로 승격되었다., 종루(鐘樓), 이현, 칠패 등에서 ...",1,None
10,generation-for-nlp-436,"(가)은/는 의병계열과 애국계몽 운동 계열의 비밀결사가 모여 결성된 조직으로, 총사...",(가)에 대한 설명으로 옳지 않은 것은?,"[공화제 국가 수립을 지향하였다., 군자금을 모집하고 친일파를 공격하였다., 북간도...",3,None
323,generation-for-nlp-819,시나리오: 후반기 자신의 고전적 조건화 실험에서 파블로프(Ivan Pavlov)의 ...,다음 시나리오를 가장 잘 설명하는 ‘고전적 조건화’(classical conditi...,"[변별, 자발적 회복, 흔적조건화, 일반화]",4,None
384,generation-for-nlp-884,시나리오: 후반기 자신의 고전적 조건화 실험에서 파블로프(Ivan Pavlov)의 ...,다음 시나리오를 가장 잘 설명하는 ‘고전적 조건화’(classical conditi...,"[차별, 자발적 회복, 흔적조건화, 일반화]",4,None
20,generation-for-nlp-449,"이제 살펴보건대, 신라가 주․군을 설치할 때 그 전정(田丁), 호구(戶口)가 현의 ...","㉠, ㉡의 거주민에 대한 설명으로 옳은 것은?","[개인의 소유물로 인정되어 매매나 증여, 상속의 대상이 되었다., 백정이라고 불렸으...",4,None
67,generation-for-nlp-507,"이제 살펴보건대, 신라가 주․군을 설치할 때 그 전정(田丁), 호구(戶口)가 현의 ...","㉠, ㉡의 거주민에 대한 설명으로 옳은 것은?","[실록청에서 사초 ․시정기․승정원일기 등을 바탕으로 실록을 편찬하였다., 임진왜란 ...",4,None


In [24]:
import pandas as pd

# 질문별로 그룹화
grouped = df.groupby('question')

# 질문은 같지만 choices 리스트 내용이 다른 경우 찾기
different_choices = []

for question, group in grouped:
    if len(group) > 1:
        # choices 리스트를 문자열로 변환하여 비교
        choices_str = group['choices'].apply(lambda x: ','.join(sorted(x)))
        if choices_str.nunique() > 1:
            different_choices.append(group)

# 결과 데이터프레임 생성
result_df = pd.concat(different_choices) if different_choices else pd.DataFrame()

# 결과 출력
if not result_df.empty:
    print("질문은 같지만 선지가 다른 경우:")
    print(result_df[['id', 'paragraph', 'question', 'choices', 'answer', 'question_plus']])
    print(f"\n총 {len(result_df)} 개의 행이 발견되었습니다.")
else:
    print("질문은 같지만 선지가 다른 경우가 없습니다.")

질문은 같지만 선지가 다른 경우:
                          id  \
27    generation-for-nlp-456   
48    generation-for-nlp-483   
13    generation-for-nlp-440   
14    generation-for-nlp-441   
12    generation-for-nlp-439   
..                       ...   
583  generation-for-nlp-1132   
584  generation-for-nlp-1133   
586  generation-for-nlp-1136   
32    generation-for-nlp-463   
60    generation-for-nlp-498   

                                             paragraph  \
27   탑골공원에 모인 수많은 학생과 시민이 독립선언식을 거행하고 만세를 부르며 거리를 행...   
48   탑골공원에 모인 수많은 학생과 시민이 독립선언식을 거행하고 만세를 부르며 거리를 행...   
13             (가) 도쿄에서 2․8 독립선언 발표 국내에서 6․10만 세 운동 발발   
14   고종은 연호를 ‘광무’로 바꾸고 환구단을 세워 이곳에서 황제로 즉위하였으며 나라 이...   
12   (가)가/이 귀산 등에게 말하기를 “세속에도 5계가 있으니, 첫째는 충성으로써 임금...   
..                                                 ...   
583  "국가적 문제가 엄중한 시기, 국가의 정의 의식으로 탄생한 국민의 양심으로 새로운 ...   
584  . . . 각 지역의 모든 사람을 통솔하는 지휘관에게 주어지는 권력은 절대군주의 권...   
586  1801년 3월 4일 피트와 보나파르트 사이의 동등한 위치를 주장하기 위해 국회의사...   
32   (가)농민군이 정부와

In [25]:
result_df

,id,paragraph,question,choices,answer,question_plus,choices_str
27,generation-for-nlp-456,탑골공원에 모인 수많은 학생과 시민이 독립선언식을 거행하고 만세를 부르며 거리를 행...,(가) 단체의 활동에 대한 설명으로 옳은 것은?,"[대동단결 선언 을 발표하였다., 국내와의 연락을 위해 교통국을 두었다., 독립군을...",2,None,"['국내와의 연락을 위해 교통국을 두었다.', '대동단결 선언 을 발표하였다.', ..."
48,generation-for-nlp-483,탑골공원에 모인 수많은 학생과 시민이 독립선언식을 거행하고 만세를 부르며 거리를 행...,(가) 단체의 활동에 대한 설명으로 옳은 것은?,"[고종이 홍범 14조를 발표하였다., 일본의 운요호가 초지진을 포격하였다., 오페르...",3,None,"['고종이 홍범 14조를 발표하였다.', '오페르트가 남연군의 묘도굴을 시도하였다...."
13,generation-for-nlp-440,(가) 도쿄에서 2․8 독립선언 발표 국내에서 6․10만 세 운동 발발,(가) 시기에 있었던 사실로 옳은 것은?,"[박상진 이 대한광복회를 조직하였다., 일제가 국가총동원법 을 적용하였다., 임병찬...",4,None,"['박상진 이 대한광복회를 조직하였다.', '상하이에서 대한민국 임시정부가 수립되었..."
14,generation-for-nlp-441,고종은 연호를 ‘광무’로 바꾸고 환구단을 세워 이곳에서 황제로 즉위하였으며 나라 이...,(가) 시기에 있었던 사실로 옳은 것은?,"[별 기군을 창설하였다 ., 교육입국조서를 발표하였다 ., 통리기무아문을 설치하였다...",4,None,"['교육입국조서를 발표하였다 .', '별 기군을 창설하였다 .', '지계 발급 사업..."
12,generation-for-nlp-439,"(가)가/이 귀산 등에게 말하기를 “세속에도 5계가 있으니, 첫째는 충성으로써 임금...",(가) 인물에 대한 설명으로 옳은 것은?,"[모든 것이 한마음에서 나온다는 일심사상을 제시하였다., 화엄사상을 연구하여 화엄일...",3,None,"['모든 것이 한마음에서 나온다는 일심사상을 제시하였다.', '왕에게 수나라에 군사..."
...,...,...,...,...,...,...,...
583,generation-for-nlp-1132,"""국가적 문제가 엄중한 시기, 국가의 정의 의식으로 탄생한 국민의 양심으로 새로운 ...",이 지문에 나타난 정보는 무엇입니까?,"[먼로 독트린에 대한 부인, 유럽 국가들이 그 지역 전체에 남아있는 식민지에서 자신...",4,None,"['먼로 독트린에 대한 부인', '미국이 서반구 전체를 보호하는 역할을 맡음으로써 ..."
584,generation-for-nlp-1133,. . . 각 지역의 모든 사람을 통솔하는 지휘관에게 주어지는 권력은 절대군주의 권...,이 지문에 나타난 정보는 무엇입니까?,"[토마스 제퍼슨 대통령의 외국인 규제법 및 선동 금지법에 대한 거부권., 제임스 먼...",4,None,['앤드류 잭슨 대통령의 1867년 재건축법 중 하나에 대한 거부권에서 인용하였습니...
586,generation-for-nlp-1136,1801년 3월 4일 피트와 보나파르트 사이의 동등한 위치를 주장하기 위해 국회의사...,이 지문에 나타난 정보는 무엇입니까?,"[관료의 규모를 비대화시킨 엽관제를 만들었습니다, 복지국가의 초석을 마련하였습니다,...",4,None,"['관료의 규모를 비대화시킨 엽관제를 만들었습니다', '도로 건설과 공공사업에 대한..."
32,generation-for-nlp-463,(가)농민군이 정부와 전주화약을 맺었다. (나) 농민군이 우금치에서 전투를 벌였다....,이를 시기 순으로 바르게 나열한 것은?,"[(다)－(가)－(라)－(나), (다)－(라)－(나)－(가), (라)－(가)－(나)...",4,None,"['(다)－(가)－(라)－(나)', '(다)－(라)－(나)－(가)', '(라)－(가..."


In [21]:
import pandas as pd

# choices 열을 문자열로 변환
df['choices_str'] = df['choices'].apply(lambda x: str(x))

# 지문과 선지(문자열로 변환된)가 동일한 그룹 생성
grouped = df.groupby(['paragraph', 'choices_str'])

# 각 그룹 내에서 질문이 다른 경우 찾기
different_questions = []

for (para, choices_str), group in grouped:
    if len(group) > 1 and group['question'].nunique() > 1:
        different_questions.append(group)

# 결과 데이터프레임 생성
result_df = pd.concat(different_questions) if different_questions else pd.DataFrame()

# 결과 출력
if not result_df.empty:
    print("지문과 선지가 같지만 질문이 다른 경우:")
    print(result_df[['id', 'paragraph', 'question', 'choices', 'answer', 'question_plus']])
    print(f"\n총 {len(result_df)} 개의 행이 발견되었습니다.")
else:
    print("지문과 선지가 같지만 질문이 다른 경우가 없습니다.")

지문과 선지가 같지만 질문이 다른 경우가 없습니다.


In [23]:
import pandas as pd

# choices를 문자열로 변환
df['choices_str'] = df['choices'].apply(lambda x: str(sorted(x)))

# 선지(문자열)와 질문이 동일한 그룹 생성
grouped = df.groupby(['choices_str', 'question'])

# 각 그룹 내에서 지문이 다른 경우 찾기
different_paragraphs = []

for (choices_str, ques), group in grouped:
    if len(group) > 1 and group['paragraph'].nunique() > 1:
        different_paragraphs.append(group)

# 결과 데이터프레임 생성
result_df = pd.concat(different_paragraphs) if different_paragraphs else pd.DataFrame()

# 결과 출력
if not result_df.empty:
    print("선지와 질문이 같지만 지문이 다른 경우:")
    print(result_df[['id', 'paragraph', 'question', 'choices', 'answer', 'question_plus']])
    print(f"\n총 {len(result_df)} 개의 행이 발견되었습니다.")
else:
    print("선지와 질문이 같지만 지문이 다른 경우가 없습니다.")

선지와 질문이 같지만 지문이 다른 경우가 없습니다.


In [5]:
import pandas as pd

# 데이터프레임이 df라고 가정합니다
target_ids = ['generation-for-nlp-819', 'generation-for-nlp-884', 'generation-for-nlp-449', 'generation-for-nlp-507']

# id 열에서 target_ids에 포함된 값을 가진 행만 선택
result_df = df[df['id'].isin(target_ids)]

# 결과 출력
result_df

,id,paragraph,question,choices,answer,question_plus
20,generation-for-nlp-449,"이제 살펴보건대, 신라가 주․군을 설치할 때 그 전정(田丁), 호구(戶口)가 현의 ...","㉠, ㉡의 거주민에 대한 설명으로 옳은 것은?","[개인의 소유물로 인정되어 매매나 증여, 상속의 대상이 되었다., 백정이라고 불렸으...",4,None
67,generation-for-nlp-507,"이제 살펴보건대, 신라가 주․군을 설치할 때 그 전정(田丁), 호구(戶口)가 현의 ...","㉠, ㉡의 거주민에 대한 설명으로 옳은 것은?","[실록청에서 사초 ․시정기․승정원일기 등을 바탕으로 실록을 편찬하였다., 임진왜란 ...",4,None
323,generation-for-nlp-819,시나리오: 후반기 자신의 고전적 조건화 실험에서 파블로프(Ivan Pavlov)의 ...,다음 시나리오를 가장 잘 설명하는 ‘고전적 조건화’(classical conditi...,"[변별, 자발적 회복, 흔적조건화, 일반화]",4,None
384,generation-for-nlp-884,시나리오: 후반기 자신의 고전적 조건화 실험에서 파블로프(Ivan Pavlov)의 ...,다음 시나리오를 가장 잘 설명하는 ‘고전적 조건화’(classical conditi...,"[차별, 자발적 회복, 흔적조건화, 일반화]",4,None


In [8]:
result_df.to_csv('result.csv', index=False)